In [2]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from datetime import datetime
import numpy as np
import json


import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [3]:
%%time
df_2021 = pd.read_csv('../전처리/wi-fi/MAC변환/presence_path_2021.csv', encoding='cp949')

CPU times: total: 57.7 s
Wall time: 59.1 s


In [4]:
%%time
df_2022 = pd.read_csv('../전처리/wi-fi/MAC변환/presence_path_2022.csv', encoding='cp949')

CPU times: total: 38.4 s
Wall time: 39.2 s


In [5]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [6]:
# 데이터프레임 타입변경
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [7]:
print('df_2021')
df_astype(df_2021)
print('-------------'*3)
print('df_2022')
df_astype(df_2022)
print('-------------'*3)

df_2021
데이터 프레임의 메모리 사용량 : 2705.92 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 1646.35 MB
감소율 : 39.2%
---------------------------------------
df_2022
데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%
---------------------------------------


In [8]:
df = df_2021[(df_2021['STARTTIMESTAMP'] >= datetime.strptime('2021-05-01 00:00:00',
                                              '%Y-%m-%d %H:%M:%S')) &
    (df_2021['STARTTIMESTAMP'] < datetime.strptime('2021-05-02 00:00:00',
                                              '%Y-%m-%d %H:%M:%S'))]

In [15]:
df = df[df['MAC_add'] == '7C573CCF497E']

In [16]:
user = len(df['CLIENTMAC'].unique())
print(user, '명')

2280 명
